## 1. LOAD VIDEO DATA

In [2]:
from trackers_simple.centroidtracker import CentroidTracker
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2


In [9]:
# Load the video
video_path = 'repte2_psiv2/data_r2/short_uab2.mp4'

# start to fill
fvs = FileVideoStream(video_path).start()
time.sleep(1.0)

# start the FPS timer
fps = FPS().start()

In [10]:
while fvs.more():
    # grab the frame from the threaded video file stream, resize
	frame = fvs.read()
	frame = imutils.resize(frame, width=450)
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	frame = np.dstack([frame, frame, frame])
	# display the size of the queue on the frame
	cv2.putText(frame, "Queue Size: {}".format(fvs.Q.qsize()),
		(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)	
	# show the frame and update the FPS counter
	cv2.imshow("Frame", frame)
	cv2.waitKey(1)
	fps.update()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
fvs.stop()

## 2. LOAD SIMPLE OBJECT TRACKER

In [3]:
# init the centroid tracker
ct = CentroidTracker()
(H, W) = (None, None)

In [ ]:
rects = []
threshold = 0.3

# loop over the frames from the video stream
while True:
    # read the next frame from the video stream and resize it
    frame = fvs.read()
    frame = imutils.resize(frame, width=450)

    # if the frame dimensions are None, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
    
    # filter weak detections
    # box rectangles
    objects = ct.update(rects)
    # loop over the tracked objects
    for (objectID, centroid) in objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
        text = "ID {}".format(objectID)
        cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break